## My bicycle reinvention in module import issue  

In [1]:
from lib import *

Pipeline = pipeline.Pipeline
DatasetProvider = provider.DatasetProvider
DatasetUploader = provider.DatasetUploader
ExpandedWindowIterator = subset_extraction.ExpandedWindowIterator
EntityIterator = subset_extraction.EntityIterator

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import scipy
from scipy import stats

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from sklearn.preprocessing import LabelEncoder

tr = transformers

In [3]:
df_provider = DatasetProvider()

In [4]:
dataset_paths = [
    '/home/denissimo/Repo/fs_project/datasets/sample_submission.csv',
    '/home/denissimo/Repo/fs_project/datasets/test.csv',
    '/home/denissimo/Repo/fs_project/datasets/shops.csv',
    '/home/denissimo/Repo/fs_project/datasets/item_categories.csv',
    '/home/denissimo/Repo/fs_project/datasets/sales_train.csv',
    '/home/denissimo/Repo/fs_project/datasets/items.csv'
]

In [5]:
df_provider.file_list = dataset_paths

datasets = df_provider.get_dataset()

In [6]:
sales = datasets['sales_train.csv']
items = datasets['items.csv']
categories = datasets['item_categories.csv']
shops = datasets['shops.csv']

In [7]:
force_category = {
    'category': {
        "PC - Гарнитуры/Наушники": "Аксессуары",
        "Игры MAC - Цифра": "Игры",
        "Игры Android - Цифра": "Игры",
        "Чистые носители (шпиль)": "Чистые носители",
        "Чистые носители (штучные)": "Чистые носители",
    },
    'shop': {
        'Интернет-магазин ЧС': 'Интернет-магазин',
        'Цифровой склад 1С-Онлайн': 'Склад',
        'Выездная Торговля': 'Выездная Торговля',
        '!Якутск Орджоникидзе, 56 фран': 'Якутск',
        '!Якутск ТЦ "Центральный" фран': 'Якутск',
    },
}

pattern = {
    'category': ' - ',
    'shop': ' ',
}

In [8]:
sales.date = sales.date.astype('datetime64[ns]')

print("Before:", sales.shape)

from datetime import date

sales = sales.loc[sales.date < np.datetime64(date(2015, 11, 1))]

sales_train = sales[
    (sales["item_cnt_day"] < 1000)
    & (sales["item_price"] > 0)
    & (sales["item_price"] < 60000)
].copy()
print("After:", sales_train.shape)

sales.head()

Before: (2935849, 6)
After: (2896778, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.00,1.0
1,2013-03-01,0,25,2552,899.00,1.0
2,2013-05-01,0,25,2552,899.00,-1.0
3,2013-06-01,0,25,2554,1709.05,1.0
4,2013-01-15,0,25,2555,1099.00,1.0


In [9]:
make_cat_name = tr.create_name_transformer(force_category['category'], pattern['category'])
make_city_name = tr.create_name_transformer(force_category['shop'], pattern['shop'])

In [10]:
data_preprocessing = {}

# Add corresponding category and shop id's to each sale
data_preprocessing['id_merging_stage'] = lambda dataset: dataset.merge(
    items, 
    on='item_id'
).merge(
    shops,
    on='shop_id'
).merge(
    categories,
    on='item_category_id'
)

# Add summary among shop_id and category_id above similar time periods (daily intervals)
data_preprocessing['summarizing_and_name_merging_stage'] = lambda dataset: dataset.groupby(
    ['date', 'date_block_num', 'shop_id', 'item_category_id', 'item_category_name', 'shop_name']
).item_cnt_day.sum().reset_index().sort_values('date')

data_preprocessing['add_generalized_names_and_encode_stage'] = lambda dataset: tr.append_columns(
    dataset=dataset, 
    columns=[
        'global_item_category_name',
        'city_name',
        'global_item_category_name_id',
        'city_id',
    ], 
    transformers=[
        lambda _dataset: _dataset["item_category_name"].apply(
            make_cat_name
        ),
        lambda _dataset: _dataset['shop_name'].apply(
            make_city_name
        ),
        lambda _dataset: LabelEncoder().fit_transform(_dataset['global_item_category_name']),
        lambda _dataset: LabelEncoder().fit_transform(_dataset['city_name']),
    ]
)

        
data_preprocessing['date_block_num_renaming'] = lambda dataset: dataset.rename(columns={'date_block_num': 'month_block'}, inplace=True)
        
data_preprocessing['date_encoding_stage'] = lambda dataset: tr.append_columns(
    dataset=dataset, 
    columns=[
        'week_block',
        'day_block',
    ], 
    transformers=[
        lambda _dataset: LabelEncoder().fit_transform(_dataset['date'].dt.to_period('W')),
        lambda _dataset: LabelEncoder().fit_transform(_dataset['date'].dt.to_period('D')),
    ]
)

data_preprocessing['create_full_matrix_stage'] = lambda _dataset: _dataset.set_index('date') \
    .groupby([
        'shop_id',
        'item_category_id',
        'month_block',
        'city_id',
        'global_item_category_name_id',
        'id'
    ]).item_cnt_day.sum() \
    .reset_index().rename(columns={'item_cnt_day': 'item_cnt_month'}) \
    .groupby(['shop_id', 'item_category_id', 'month_block', 'city_id', 'global_item_category_name_id', 'id']).item_cnt_month.sum().reset_index() \
    .groupby(['month_block', 'id']).item_cnt_month.sum().unstack().fillna(0) \
    .stack().reset_index().rename(columns={0:'item_cnt_month'})

In [11]:
pipeline = Pipeline(
    tasks=data_preprocessing, 
    task_queue = [
        'id_merging_stage',
        'summarizing_and_name_merging_stage',
        'add_generalized_names_and_encode_stage',
        'date_block_num_renaming',
        'date_encoding_stage',
    ]
)

In [12]:
pipeline_test = pipeline(sales)
pipeline_train = pipeline(sales_train)

_ = pipeline_test.proceed_all()
_ = pipeline_train.proceed_all()

Stage - id_merging_stage complete
Stage - summarizing_and_name_merging_stage complete
Stage - add_generalized_names_and_encode_stage complete
Stage - date_block_num_renaming complete
Stage - date_encoding_stage complete
Stage - id_merging_stage complete
Stage - summarizing_and_name_merging_stage complete
Stage - add_generalized_names_and_encode_stage complete
Stage - date_block_num_renaming complete
Stage - date_encoding_stage complete


In [13]:
task_df = {}

task_df['test'] = pipeline_test.result_storage['date_encoding_stage']
task_df['train'] = pipeline_train.result_storage['date_encoding_stage']

In [14]:
idx = task_df['test'].loc[:,['city_id', 'global_item_category_name_id', 'city_name', 'global_item_category_name']].value_counts().sort_index()
idx = pd.DataFrame({'id': [i for i in range(idx.size)]}, idx.index)
idx.reset_index(inplace=True)

In [15]:
task_df['test'] = task_df['test'].merge(idx, on=['city_id', 'global_item_category_name_id', 'city_name', 'global_item_category_name'])
task_df['train'] = task_df['train'].merge(idx, on=['city_id', 'global_item_category_name_id', 'city_name', 'global_item_category_name'])

In [16]:
idx['pair_name'] = idx['city_name'] + ' - ' + idx['global_item_category_name']

In [17]:
task_df_copy = {}

task_df_copy['train'] = task_df['train'].copy()
task_df['test']['pair_name'] = task_df['test']['city_name'] + ' - ' + task_df['test']['global_item_category_name']

task_df['train'] = data_preprocessing['create_full_matrix_stage'](task_df['train'])

In [18]:
task_df['train']

,month_block,id,item_cnt_month
0,0,0,66.0
1,0,1,0.0
2,0,2,20.0
3,0,3,240.0
4,0,4,263.0
...,...,...,...
14887,33,433,87.0
14888,33,434,98.0
14889,33,435,14.0
14890,33,436,12.0


In [19]:
sample_df = task_df['train'][task_df['train'].id == 3]

In [20]:
CONFIG = {
    'validation_window': {
        'min': 24,
        'max': 31,
    },
    'training_window': {
        'min': 16
    }
}

In [24]:
def add_aggregation(feature_set, series, name):
    feature_set[name + '_mean'] = series.mean()
    feature_set[name + '_std'] = series.std()

def target_extractor(month):
    return task_df['train'][task_df['train'].month_block == month][['id', 'item_cnt_month']].set_index('id').sort_index()

In [25]:
iteration_rule = ExpandedWindowIterator('month_block', target_extractor, min_idx=CONFIG['validation_window']['min'], step=1, max_idx=CONFIG['validation_window']['max'])

In [26]:
subset_extractor = EntityIterator('id')

In [30]:
def generate_feature_row(split_x, target, window, _id):
    
    _feature_set = {}
    _serieses = {}

    _feature_set['window'] = window
    _feature_set['id'] = _id

    _sale_series = split_x.set_index('month_block').item_cnt_month

    _serieses[f'lag_1'] = tr.lag(_sale_series, 1, 0)
    _serieses['diff_1'] = tr.diff(_sale_series, 1)
    _serieses['diff_2'] = tr.diff(_sale_series, 2)

    for i in range(1,13):
        _feature_set[f'lag_{i}'] = _serieses['lag_1'].to_list()[-i]

    for i in range(1,7):
        _feature_set[f'diff_1_{i}'] = _serieses['diff_1'].to_list()[-i]

    for i in range(1,7):
        _feature_set[f'diff_2_{i}'] = _serieses['diff_2'].to_list()[-i]
    
    add_aggregation(_feature_set, _sale_series, '')
    add_aggregation(_feature_set, _sale_series[-12:], 'last_12')
    add_aggregation(_feature_set, _sale_series[-6:], 'last_6')
    add_aggregation(_feature_set, _sale_series[-2:], 'last_2')
    add_aggregation(_feature_set, _serieses['diff_1'], 'diff_1')
    add_aggregation(_feature_set, _serieses['diff_2'], 'diff_2')
    add_aggregation(_feature_set, _serieses['diff_1'][-12:], 'diff_1_last_12')
    add_aggregation(_feature_set, _serieses['diff_1'][-6:], 'diff_1_last_6')
    add_aggregation(_feature_set, _serieses['diff_1'][-2:], 'diff_1_last_2')
    _feature_set['last_observed'] = _sale_series.values[-1]
    _feature_set['time'] = _sale_series.index[-1]
    _feature_set['month_number'] = _feature_set['time'] % 12
    _feature_set['target'] = target

    return pd.DataFrame(_feature_set.items()).set_index(0).transpose()

In [31]:
train = None

for window, split_x, split_y in iteration_rule(task_df['train']):
    as_list = split_y.item_cnt_month.to_list()
    for _id, subset in subset_extractor(split_x):
        new_feature = generate_feature_row(subset, as_list[_id], window, _id)
        if train is None:
            train = new_feature
        else:
            train.append(new_feature)
        

In [32]:
train

,window,id,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,...,diff_1_last_12_mean,diff_1_last_12_std,diff_1_last_6_mean,diff_1_last_6_std,diff_1_last_2_mean,diff_1_last_2_std,last_observed,time,month_number,target
1,16.0,0.0,34.0,43.0,75.0,56.0,113.0,34.0,31.0,33.0,...,1.083333,32.497436,0.166667,46.812035,-4.0,7.071068,35.0,16.0,4.0,35.0
